In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.krylovspace import CG, GMRes
from ngsolve.webgui import Draw
from libbem import *

In [ ]:
largebox = Box ((-2,-2,-2), (2,2,2) )
b1 = Box ( (-1,-1,0.5), (1,1,1) )
b2 = Box ( (-1,-1,-1), (1,1,-0.5))

b1.name = "top"
b2.name = "bot"
outer = largebox-b1-b2
outer.name = "air"
largebox.faces.name = "outer"

shape = Compound( [b1,b2,outer] )
mesh = Mesh(OCCGeometry(shape).GenerateMesh (maxh=1))
# mesh = Mesh(OCCGeometry(largebox).GenerateMesh (maxh=1))

Draw (mesh);

Only FEM:

In [ ]:
order = 2
fesH1 = H1(mesh, order=order)
u,v = fesH1.TnT()
a = BilinearForm(grad(u)*grad(v)*dx + 1e-5*u*v*dx).Assemble()
source = mesh.MaterialCF( { "top" : 1, "bot" : -1 }, default=0)
# source = exp ( -4 * ( x**2+y**2+(z-1)**2 ) ) - exp ( -4 * ( x**2+y**2+(z+1)**2 ) ) 
f = LinearForm(source*v*dx(bonus_intorder=10)).Assemble()

gfu = GridFunction(fesH1)
gfu.vec[:] = a.mat.Inverse() * f.vec
Draw (gfu, clipping={"y":1, "z":0, "dist":0.0, "function" : True });

In [ ]:
fesL2 = SurfaceL2(mesh, order=order-1, dual_mapping=True, definedon=mesh.Boundaries("outer"))
f2 = LinearForm(fesL2).Assemble()  # 0-vector
print ("ndof = ", fesL2.ndof)

In [ ]:
with TaskManager():
    V = SingleLayerPotentialOperator(fesL2, intorder=12, leafsize=40, eta=3., eps=1e-4, method="svd", testhmatrix=False)
    K = DoubleLayerPotentialOperator(fesH1, fesL2, 
                                     trial_definedon=mesh.Boundaries("outer"),  test_definedon=mesh.Boundaries("outer"), 
                                     intorder=12, leafsize=40, eta=3., eps=1e-4, method="svd", testhmatrix=False)
    S = HypersingularOperator(fesH1, definedon=mesh.Boundaries("outer"), intorder=12, leafsize=40, eta=3., eps=1e-4, method="svd", testhmatrix=False)
    Id = BilinearForm(fesH1.TrialFunction()*fesL2.TestFunction().Trace()*ds("outer")).Assemble()
    

In [ ]:
lhs = BlockMatrix ( [ [ a.mat+S.mat, (0.5*Id.mat+K.mat).T ], [ (0.5*Id.mat+K.mat), -V.mat ] ] )
rhs = BlockVector( [ f.vec, f2.vec ] )

bndmass = BilinearForm( fesL2.TrialFunction().Trace()*fesL2.TestFunction().Trace()*ds).Assemble()
pre = BlockMatrix ( [ [a.mat.Inverse(), None], [None, bndmass.mat.Inverse(freedofs=fesL2.FreeDofs())] ])

In [ ]:
sol = GMRes(A=lhs, b=rhs, pre=pre, maxsteps=200)

In [ ]:
gfu.vec[:] = sol[0]
Draw (gfu, clipping={"y":1, "z":0, "dist":0.0, "function" : True }); 